# Calculate SPG from CBS html



In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

import os
import sys

sys.path.append(r'C:\Users\lenha\Repos\phi-utils')

# N.B. this was has not been merged to main in that repo....
from philosofool.data_sources.utils import read_yml
#os.path.exists(os.path.norm('C:Users/lenha/Repos/phi-utils'))

from fantasy_baseball_draft.spg import (
    spgs_from_standings_html as calc_spg, 
    FantasyValuator
)
from fantasy_baseball_draft.stats import hitter_fwar, pitcher_fwar
from fantasy_baseball_draft.utils import process_players, process_hitters, load_cbs_data, free_agents, StatSynonyms

In [ ]:
data_path = read_yml('local/config.yml')['paths']['local_data']
standings_path = os.path.join(data_path, r'standings\cbs_2021_standings.html')

spg = calc_spg(standings_path)

In [ ]:
projections_path = os.path.join(data_path, r'projections')
eligibility = load_cbs_data(os.path.join(data_path, 'eligibility/eligibility.csv'))
hitters = (
    load_cbs_data(os.path.join(projections_path, 'cbs_hitters.csv'))
    .pipe(process_hitters, eligibility)
    .pipe(process_players)
    .drop(columns='player')
)
pitchers = (
    load_cbs_data(os.path.join(projections_path, 'cbs_pitchers.csv'))
    .pipe(process_players)
    .drop(columns='player')
)
free_agents(pitchers).sum()
pitchers.head()

In [ ]:
from fantasy_baseball_draft.spg import ReplacementValuator
valuator = FantasyValuator(spg)
valuator.valuate_hitters(hitters, 16*11).hist()
valuator.valuate_pitchers(pitchers, 16*11).hist()
replacement = ReplacementValuator(spg)
#replacement.valuate_hitters(hitters, 160).hist()
#replacement.find_position_values(hitters, 160)

In [ ]:
from fantasy_baseball_draft.stats import hitter_fwar, pitcher_fwar
hitter_fwar(hitters, spg['spg'], spg['median']).sort_values()[-180:]